In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from vnstock import *

In [8]:
START_DATE = '2013-01-01'
END_DATE = '2023-10-12'
SYSBOL = 'SSI'

In [20]:
# Lấy dữ liệu
data = stock_historical_data(SYSBOL, START_DATE, END_DATE, "1D", "stock")

data = data.rename(columns={'time' : 'Date', 'close' : 'Close'})

data['Date'] = pd.to_datetime(data['Date'])
data = data.set_axis(data['Date'])
data = data.drop(columns=['open', 'high', 'low', 'volume', 'ticker'])


# Chuẩn bị dữ liệu
data = data.dropna()
# data = data.set_index("Date")


data.head()

data = data[["Close"]]


In [21]:
# Tạo bộ dữ liệu huấn luyện và kiểm tra
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size

train_data = data[:train_size]
test_data = data[train_size:]

In [23]:
# Tạo bộ dữ liệu chuẩn hóa
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Tạo mô hình LSTM
look_back = 15
model = Sequential()
model.add(LSTM(128, input_shape=(look_back,1)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Huấn luyện mô hình
num_epochs = 100
model.compile(optimizer="adam", loss="mse")
model.fit_generator(train_data, epochs=num_epochs)

# Dự đoán điểm giao dịch
predictions = model.predict(test_data)
predictions = scaler.inverse_transform(predictions)

Epoch 1/100


/tmp/ipykernel_2198/2273357524.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, epochs=num_epochs)


ValueError: in user code:

    File "/home/hieuhv10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/hieuhv10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/hieuhv10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/hieuhv10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1128, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/hieuhv10/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1082, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=mse, and therefore expects target d

In [12]:




# Tìm điểm giao dịch
buy_signals = []
sell_signals = []
for i in range(len(predictions)):
  if predictions[i] > predictions[i - 1]:
    buy_signals.append(test_data.index[i])
  elif predictions[i] < predictions[i - 1]:
    sell_signals.append(test_data.index[i])

# Tính toán lợi nhuận
profit = 0
for i in range(len(buy_signals)):
  profit += predictions[i] - test_data["Close"].iloc[buy_signals[i]]

# In kết quả
print("Lợi nhuận:", profit)
print("Điểm mua:", buy_signals)
print("Điểm bán:", sell_signals)

ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 1)